In [31]:
content='```json\n{ "questions": [\n        {\n            "question": "What is a blind interview?",\n            "answers": [\n                    {\n                        "answer": "An interview conducted in complete darkness",\n                        "correct": false\n                    },\n                    {\n                        "answer": "An interview where the interviewers are blind",\n                        "correct": true\n                    },\n                    {\n                        "answer": "An interview without any questions",\n                        "correct": false\n                    },\n                    {\n                        "answer": "An interview with no candidates",\n                        "correct": false\n                    },\n            ]\n        },\n                    {\n            "question": "Why do some people support blind interviews?",\n            "answers": [\n                    {\n                        "answer": "Because they are conducted in secret",\n                        "correct": false\n                    },\n                    {\n                        "answer": "Because they save time by only focusing on relevant skills",\n                        "correct": true\n                    },\n                    {\n                        "answer": "Because they are more entertaining",\n                        "correct": false\n                    },\n                    {\n                        "answer": "Because they are easier to cheat on",\n                        "correct": false\n                    },\n            ]\n        },\n                    {\n            "question": "What kind of information do interviewers consider in blind interviews?",\n            "answers": [\n                    {\n                        "answer": "Place of birth and family relations",\n                        "correct": false\n                    },\n                    {\n                        "answer": "Skills relevant to the job",\n                        "correct": true\n                    },\n                    {\n                        "answer": "Favorite color and food",\n                        "correct": false\n                    },\n                    {\n                        "answer": "Favorite TV shows and movies",\n                        "correct": false\n                    },\n            ]\n        }\n    ]\n }\n```'

In [40]:
import json

content = '''
{ "questions": 
        {
            "aasdfasd"
        },
 }
'''

# Print the content before attempting to load it as JSON
print(content)

# Remove the triple backticks and 'json' part from the content string
content = content.replace('```json\n', '').replace('```', '').replace('\n','')

# Load the string as JSON
data = json.loads(content)

# Write the JSON data to a file
with open('output.json', 'w') as json_file:
    json.dump(data, json_file, indent=4)

print("JSON data has been written to 'output.json'")



{ "questions": 
        {
            "question": "What is a blind interview?",
            "answers": [
                    {
                        "answer": "An interview conducted in complete darkness",
                        "correct": false
                    },
                    {
                        "answer": "An interview where the interviewers are blind",
                        "correct": true
                    },
                    {
                        "answer": "An interview without any questions",
                        "correct": false
                    },
                    {
                        "answer": "An interview with no candidates",
                        "correct": false
                    },
            ]
        },
 }



JSONDecodeError: Expecting value: line 1 column 746 (char 745)